# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 7 2023 1:22PM,256843,10,Clear-9128136,"ClearSpec, LLC",Released
1,Feb 7 2023 1:22PM,256843,10,Clear-9128137,"ClearSpec, LLC",Released
2,Feb 7 2023 1:22PM,256843,10,Clear-9128640,"ClearSpec, LLC",Released
3,Feb 7 2023 1:22PM,256843,10,Clear-9128641,"ClearSpec, LLC",Released
4,Feb 7 2023 1:22PM,256843,10,Clear-9128661,"ClearSpec, LLC",Released
5,Feb 7 2023 1:22PM,256843,10,Clear-9128664,"ClearSpec, LLC",Released
6,Feb 7 2023 1:22PM,256843,10,Clear-9128665,"ClearSpec, LLC",Released
7,Feb 7 2023 1:22PM,256843,10,Clear-9128666,"ClearSpec, LLC",Released
8,Feb 7 2023 1:22PM,256843,10,Clear-9128668,"ClearSpec, LLC",Released
9,Feb 7 2023 1:22PM,256843,10,Clear-9128672,"ClearSpec, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,256839,Released,1
36,256840,Released,1
37,256841,Released,4
38,256842,Released,1
39,256843,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256839,NaN,NaN,1.0
256840,NaN,NaN,1.0
256841,NaN,NaN,4.0
256842,NaN,NaN,1.0
256843,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256777,0.0,0.0,1.0
256781,0.0,0.0,1.0
256782,0.0,84.0,21.0
256788,0.0,0.0,1.0
256792,2.0,9.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256777,0,0,1
256781,0,0,1
256782,0,84,21
256788,0,0,1
256792,2,9,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256777,0,0,1
1,256781,0,0,1
2,256782,0,84,21
3,256788,0,0,1
4,256792,2,9,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256777,,,1
1,256781,,,1
2,256782,,84,21
3,256788,,,1
4,256792,2,9,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 7 2023 1:22PM,256843,10,"ClearSpec, LLC"
13,Feb 7 2023 1:20PM,256841,12,Hush Hush
17,Feb 7 2023 1:20PM,256842,10,"Senseonics, Incorporated"
18,Feb 7 2023 1:13PM,256840,21,"NBTY Global, Inc."
19,Feb 7 2023 1:11PM,256839,10,Emerginnova
20,Feb 7 2023 1:11PM,256838,10,"Snap Medical Industries, LLC"
22,Feb 7 2023 1:10PM,256837,10,ISDIN Corporation
35,Feb 7 2023 1:10PM,256835,10,ISDIN Corporation
56,Feb 7 2023 1:07PM,256834,16,Sartorius Stedim North America
57,Feb 7 2023 12:58PM,256833,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 7 2023 1:22PM,256843,10,"ClearSpec, LLC",,,13
1,Feb 7 2023 1:20PM,256841,12,Hush Hush,,,4
2,Feb 7 2023 1:20PM,256842,10,"Senseonics, Incorporated",,,1
3,Feb 7 2023 1:13PM,256840,21,"NBTY Global, Inc.",,,1
4,Feb 7 2023 1:11PM,256839,10,Emerginnova,,,1
5,Feb 7 2023 1:11PM,256838,10,"Snap Medical Industries, LLC",,,2
6,Feb 7 2023 1:10PM,256837,10,ISDIN Corporation,,,13
7,Feb 7 2023 1:10PM,256835,10,ISDIN Corporation,,,21
8,Feb 7 2023 1:07PM,256834,16,Sartorius Stedim North America,,1,
9,Feb 7 2023 12:58PM,256833,10,"Methapharm, Inc.",,,45


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 1:22PM,256843,10,"ClearSpec, LLC",13,,
1,Feb 7 2023 1:20PM,256841,12,Hush Hush,4,,
2,Feb 7 2023 1:20PM,256842,10,"Senseonics, Incorporated",1,,
3,Feb 7 2023 1:13PM,256840,21,"NBTY Global, Inc.",1,,
4,Feb 7 2023 1:11PM,256839,10,Emerginnova,1,,
5,Feb 7 2023 1:11PM,256838,10,"Snap Medical Industries, LLC",2,,
6,Feb 7 2023 1:10PM,256837,10,ISDIN Corporation,13,,
7,Feb 7 2023 1:10PM,256835,10,ISDIN Corporation,21,,
8,Feb 7 2023 1:07PM,256834,16,Sartorius Stedim North America,,1,
9,Feb 7 2023 12:58PM,256833,10,"Methapharm, Inc.",45,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 1:22PM,256843,10,"ClearSpec, LLC",13,,
1,Feb 7 2023 1:20PM,256841,12,Hush Hush,4,,
2,Feb 7 2023 1:20PM,256842,10,"Senseonics, Incorporated",1,,
3,Feb 7 2023 1:13PM,256840,21,"NBTY Global, Inc.",1,,
4,Feb 7 2023 1:11PM,256839,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 1:22PM,256843,10,"ClearSpec, LLC",13.0,NaN,NaN
1,Feb 7 2023 1:20PM,256841,12,Hush Hush,4.0,NaN,NaN
2,Feb 7 2023 1:20PM,256842,10,"Senseonics, Incorporated",1.0,NaN,NaN
3,Feb 7 2023 1:13PM,256840,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Feb 7 2023 1:11PM,256839,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 7 2023 1:22PM,256843,10,"ClearSpec, LLC",13.0,0.0,0.0
1,Feb 7 2023 1:20PM,256841,12,Hush Hush,4.0,0.0,0.0
2,Feb 7 2023 1:20PM,256842,10,"Senseonics, Incorporated",1.0,0.0,0.0
3,Feb 7 2023 1:13PM,256840,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Feb 7 2023 1:11PM,256839,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5906816,240.0,12.0,2.0
12,256841,4.0,0.0,0.0
15,256782,21.0,84.0,0.0
16,513639,1.0,1.0,0.0
19,770475,14.0,0.0,0.0
20,770435,2.0,10.0,0.0
21,770455,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5906816,240.0,12.0,2.0
1,12,256841,4.0,0.0,0.0
2,15,256782,21.0,84.0,0.0
3,16,513639,1.0,1.0,0.0
4,19,770475,14.0,0.0,0.0
5,20,770435,2.0,10.0,0.0
6,21,770455,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,240.0,12.0,2.0
1,12,4.0,0.0,0.0
2,15,21.0,84.0,0.0
3,16,1.0,1.0,0.0
4,19,14.0,0.0,0.0
5,20,2.0,10.0,0.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,240.0
1,12,Released,4.0
2,15,Released,21.0
3,16,Released,1.0
4,19,Released,14.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,12.0,0.0,84.0,1.0,0.0,10.0,0.0
Released,240.0,4.0,21.0,1.0,14.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,84.0,1.0,0.0,10.0,0.0
2,Released,240.0,4.0,21.0,1.0,14.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,12.0,0.0,84.0,1.0,0.0,10.0,0.0
2,Released,240.0,4.0,21.0,1.0,14.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()